# CS2 Event dataset

Create CS2 event dataset from the heterogeneous shapshot graphs. Features to keep track of (P - player, M - map, U - universal):

In [5]:
import torch

import pandas as pd
import polars as pl
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

import time
import json
import sys
import os

pd.set_option('display.max_columns', 100)

pl.Config.set_tbl_rows(10)

sys.path.append(os.path.abspath('../../../package'))

### Features to examine:


  - P. position changes (X, Y, Z)
  - P. velocity changes (X, Y, Z)
  - P. look direction changes
  - P. travelled to new map area
  - P. entered bombsite
  - P. left bombsite
  - P. entered active contact spot
  - P. left active contact spot
  - P. actual map area

  - P. changed to primary weapon
  - P. switched primary weapon
  - P. primary weapon
  - P. changed to secondary weapon
  - P. switched secondary weapon
  - P. secondary weapon
  - P. is shooting / throwing a grenade
  - P. got low on ammunition

  - P. dealt damage
  - P. eliminated an opponent
  - P. lost health
  - P. lost armor
  - P. got eliminated

  - P. was spotted
  - P. got flashed
  - P. started reloading
  - P. finished reloading
  - P. scoped in 
  - P. scoped out
  - P. started defusing

  - M. map part got set on fire
  - M. map part molotov extinguished
  - M. molotovs burning on map part
  - M. map part got smoked
  - M. map part smoke ended
  - M. active smokes on map part

  - U. bomb got dropped
  - U. bomb was picked up
  - U. bomb getting planted
  - U. bomb got planted
  - U. bombsite the bomb was planted on (A or B)
  - U. bomb position in the 9x9 matrix

  - PRED. predicted win probability for the snapshot

### Dataset creation